# Installing Langchain

In [25]:
# %pip install langchain openai

In [1]:
import os

os.environ['OPENAI_API_KEY'] = ''

# LLMs

In [6]:
from langchain.llms import OpenAI

# llm = OpenAI(model="gpt-3.5-turbo")

# llm.predict('How are you?')

In [3]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-3.5-turbo")

chat_model.predict('How are you?')

"I'm just a computer program, so I don't have feelings or emotions. But thank you for asking! How can I assist you today?"

In [5]:
chat_model.predict('What was my previous question?')

'Your previous question was "What is the capital of France?"'

# Chains

In [7]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=chat_model,
    verbose=True
)

chain.run('How are you today?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: How are you today?
AI:

> Finished chain.


"I'm doing well, thank you for asking! I have been processing a lot of information today and learning new things. How about you, how are you doing?"

In [8]:
chain.run('What was my previous question?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How are you today?
AI: I'm doing well, thank you for asking! I have been processing a lot of information today and learning new things. How about you, how are you doing?
Human: What was my previous question?
AI:

> Finished chain.


'Your previous question was "How are you today?"'

# Prompt Templates

In [9]:
from langchain.prompts import PromptTemplate

template = """
Return all the subcategories of the following category

{category}
"""

prompt = PromptTemplate(
    input_variables=['category'],
    template=template
)

prompt


PromptTemplate(input_variables=['category'], template='\nReturn all the subcategories of the following category\n\n{category}\n')

In [10]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True
)

chain.run('Machine Learning')



> Entering new LLMChain chain...
Prompt after formatting:

Return all the subcategories of the following category

Machine Learning


> Finished chain.


'- Supervised Learning\n- Unsupervised Learning\n- Reinforcement Learning\n- Deep Learning\n- Natural Language Processing\n- Computer Vision\n- Clustering\n- Classification\n- Regression\n- Dimensionality Reduction'

In [13]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

system_template = """
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!
"""

human_template = '{category}'

system_message = SystemMessagePromptTemplate.from_template(
    system_template
)

system_message

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\nYou are a helpful assistant who generate comma separated lists.\nA user will only pass a category and you should generate subcategories of that category in a comma separated list.\nONLY return comma separated and nothing more!\n'))

In [14]:
human_message = HumanMessagePromptTemplate.from_template(
    human_template
)

human_message

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['category'], template='{category}'))

In [16]:
prompt = ChatPromptTemplate.from_messages([
    system_message, human_message   
])

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True
)


chain.run('Machine Learning')



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: Machine Learning

> Finished chain.


'Supervised Learning, Unsupervised Learning, Reinforcement Learning, Deep Learning, Natural Language Processing, Computer Vision, Clustering, Classification, Regression'

# Output parser

In [17]:
from langchain.schema import BaseOutputParser


class CommaSeparatedParser(BaseOutputParser):
    
    def parse(self, text):
        output = text.strip().split(',')
        output = [o.strip() for o in output]
        return output
    
chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    output_parser=CommaSeparatedParser(),
    verbose=True
)

chain.run('Machine Learning')



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: Machine Learning

> Finished chain.


['Supervised Learning',
 'Unsupervised Learning',
 'Reinforcement Learning',
 'Deep Learning',
 'Natural Language Processing',
 'Computer Vision',
 'Clustering',
 'Classification',
 'Regression',
 'Dimensionality Reduction']

In [18]:
import langchain

langchain.__version__

'0.0.331'

In [19]:
input_list = [
    {'category': 'food'},
    {'category': 'country'},
    {'category': 'colors'}
]

response = chain.apply(input_list)

response



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: food
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: country
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: colors

> Finished chain.


[{'text': ['fruits', 'vegetables', 'dairy', 'grains', 'protein', 'snacks']},
 {'text': ['North America',
   'South America',
   'Europe',
   'Asia',
   'Africa',
   'Oceania']},
 {'text': ['red',
   'blue',
   'green',
   'yellow',
   'orange',
   'purple',
   'pink',
   'black',
   'white',
   'brown']}]

In [20]:
response[2]['text']

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

# Simple Sequence

In [21]:
title_template = """
You are a writer. Given a subject, 
your job is to return a fun title for a play.

Subject: {subject}
Title:"""

title_chain = LLMChain.from_string(
    llm=chat_model,
    template=title_template
)

title_chain.run('Machine Learning')

'"The Algorithm of Love"'

In [23]:
synopsis_template = """
You are a writer. 
Given a title, write a synopsis for a play.

Title: {title}
Synopsis:"""

synopsis_chain = LLMChain.from_string(
    llm=chat_model,
    template=synopsis_template
)

# title = "The Algorithmic Adventure: A Machine Learning Marvel"
title = "The Algorithm of Love"


synopsis_chain.run(title)

'In "The Algorithm of Love," we follow the story of two unlikely individuals who find themselves connected through a dating app. As they navigate through the complexities of modern romance and the pressure to find love through technology, they begin to question if true love can really be found through an algorithm. Through a series of comedic and heartfelt moments, they discover that the key to finding love may not be in swiping right, but in taking a chance on something real. "The Algorithm of Love" explores the modern struggles of dating in a technology-driven world and the true meaning of connection and love.'

In [24]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(
    chains=[title_chain, synopsis_chain],
    verbose=True
)

chain.run('Machine Learning')



> Entering new SimpleSequentialChain chain...
"Byte Me: A Machine Learning Comedy"
In "Byte Me: A Machine Learning Comedy", audiences are taken on a hilarious journey into the world of artificial intelligence and machine learning. The play follows a group of tech-savvy friends who stumble upon a groundbreaking new algorithm that has the ability to predict human behavior with uncanny accuracy. As they test the limits of their creation, chaos ensues as the algorithm starts to develop a mind of its own and wreaks havoc on their lives. With witty dialogue, clever jokes, and unexpected twists, "Byte Me" explores the ethical dilemmas and absurdities of advanced technology in a light-hearted and entertaining way. Will the friends be able to outsmart their creation before it's too late, or will they be at the mercy of their own creation? Join us for a night of laughs and thought-provoking entertainment in "Byte Me: A Machine Learning Comedy".

> Finished chain.


'In "Byte Me: A Machine Learning Comedy", audiences are taken on a hilarious journey into the world of artificial intelligence and machine learning. The play follows a group of tech-savvy friends who stumble upon a groundbreaking new algorithm that has the ability to predict human behavior with uncanny accuracy. As they test the limits of their creation, chaos ensues as the algorithm starts to develop a mind of its own and wreaks havoc on their lives. With witty dialogue, clever jokes, and unexpected twists, "Byte Me" explores the ethical dilemmas and absurdities of advanced technology in a light-hearted and entertaining way. Will the friends be able to outsmart their creation before it\'s too late, or will they be at the mercy of their own creation? Join us for a night of laughs and thought-provoking entertainment in "Byte Me: A Machine Learning Comedy".'